## White Box on MNIST (Table 3)

In [1]:
from tabulate import tabulate
table = []
DEFENSES = ['Feature squeezing',
               'Label smoothing',
               'FGSM Adversrial Training',
               'VAT',
               'Gaussian + BRELU ',
               'Gaussian + RELU',
               'Vanilla CNN'
           ]
for d in range(len(DEFENSES)):
    table.append([DEFENSES[d]])

In [2]:
import numpy as np

import json
import os, sys
from os.path import join, exists

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from config import DATA_PATH

### White box on adversarial examples frm FGSM $(\epsilon = 0.1)$, MNIST

Defences

In [3]:

CLASSIFIERS = ["cnn/relu/featsqueeze1", #Feature Squeezing
               "cnn/relu/labsmooth", # Label Smoothing
               "cnn/relu/adv-trained/fgsm/eps0.30_train.npy", # Adversarial training FGSM eps=0.4
               "cnn/relu/adv-trained/vat/eps2.10_train.npy",  # Adversarial Training VAT eps=2.1
               "cnn/brelu/gaussian/stdev0.30/pert-insts10", # Gaussian sigma =0.3 + brelu
               "cnn/relu/gaussian/stdev0.30/pert-insts10",  # Gaussian sigma =0.3 + relu
               "cnn/relu"]  # vanilla cnn-relu

In [4]:
INPUT_PATH = join(DATA_PATH, 'classifiers','mnist')
for c in CLASSIFIERS:
    assert exists(join(INPUT_PATH, c, "best-weights.h5"))
    assert exists(join(INPUT_PATH, c, "model.json"))
    assert exists(join(INPUT_PATH, c, "params.json"))

Adversarial Images

In [5]:
WBox_ATTACKS = join(DATA_PATH,"adversarial",'mnist')

ADV_IMAGES = ["cnn/relu/featsqueeze1", #Feature Squeezing
               "cnn/relu/labsmooth", # Label Smoothing
               "cnn/relu/adv-trained/", # Adversarial training FGSM eps=0.4
               "cnn/relu/adv-trained/vat/eps2.10_train",  # Adversarial Training VAT eps=2.1
               "cnn/brelu/gaussian/stdev0.30/pert-insts10", # Gaussian sigma =0.3 + brelu
               "cnn/relu/gaussian/stdev0.30/pert-insts10",  # Gaussian sigma =0.3 + relu
               "cnn/relu"]  # vanilla cnn-relu
for c in ADV_IMAGES[0:]:
    #print(c)
    assert exists(join(WBox_ATTACKS, c, "fgsm", "eps0.10_test.npy"))
    #assert exists(join(WBox_ATTACKS, c, "deepfool", "test.npy"))

In [6]:
# Load Classifiers
from src.classifiers.cnn import CNN
from src.classifiers.utils import load_classifier
from src.utils import load_mnist
import keras.backend as K
import tensorflow as tf

#important for ensuring the model_weights are loaded in the same session as the one where metrics are computed
session = tf.Session()
K.set_session(session)

Using TensorFlow backend.


In [7]:
(X_train,_), (X_test, Y_test) = load_mnist() 

### Accuracies

In [8]:
from src.metrics import accuracy_at_mp

params = {"eps_step": 0.05,
        "eps_max": 1.0,
        "clip_min": 0.0,
        "clip_max": 1.0}

for c,a,d in zip(CLASSIFIERS,ADV_IMAGES,range(len(DEFENSES))):
    classifier = load_classifier(join(INPUT_PATH, c),'best-weights.h5')
    avg_nn_dist = accuracy_at_mp(X_test, Y_test, classifier.model, session, "fgsm", params)
    
    temp = table[d]
    temp.append(avg_nn_dist)
    table[d] = temp
    print(DEFENSES[d],avg_nn_dist)

1
Flips wrt: true  3561  pred  3390
 9888/10000 [============================>.] - ETA: 0sFeature squeezing 35.61
Flips wrt: true  3706  pred  3547
 9888/10000 [============================>.] - ETA: 0sLabel smoothing 37.06
Flips wrt: true  5644  pred  5544
 9888/10000 [============================>.] - ETA: 0sFGSM Adversrial Training 56.44
Flips wrt: true  7332  pred  7251
10000/10000 [==============================] - 3s     
VAT 73.32
Flips wrt: true  6956  pred  6919
 9888/10000 [============================>.] - ETA: 0sGaussian + BRELU  69.56
Flips wrt: true  7396  pred  7334
 9888/10000 [============================>.] - ETA: 0sGaussian + RELU 73.96
Flips wrt: true  5207  pred  5129
 9888/10000 [============================>.] - ETA: 0sVanilla CNN 52.07


### Robustness

In [9]:
from src.metrics import empirical_robustness

params = {"eps_step": 0.05,
        "eps_max": 1.0,
        "clip_min": 0.0,
        "clip_max": 1.0}
for c,a,d in zip(CLASSIFIERS,ADV_IMAGES,range(len(DEFENSES))):
    classifier = load_classifier(join(INPUT_PATH, c),'best-weights.h5')
    emp_robust = empirical_robustness(X_test, classifier.model, session, "fgsm", params)
    
    temp = table[d]
    temp.append(emp_robust)
    table[d] = temp
    print(DEFENSES[d],emp_robust)
'''
cnn/relu/featsqueeze1 0.161807
cnn/relu/labsmooth 0.183515
cnn/relu/adv-trained/fgsm/eps0.30_train.npy 0.132598
cnn/relu/adv-trained/vat/eps2.10_train.npy 0.0876128
cnn/brelu/gaussian/stdev0.30/pert-insts10 0.228296
cnn/relu/gaussian/stdev0.30/pert-insts10 0.0665115
cnn/relu 0.177879
'''

1
6610
Feature squeezing 0.143702
6453
Label smoothing 0.152006
4456
FGSM Adversrial Training 0.226028
2749
VAT 0.308226
3081
Gaussian + BRELU  0.471461
2666
Gaussian + RELU 0.44477
4871
Vanilla CNN 0.202975


'\ncnn/relu/featsqueeze1 0.161807\ncnn/relu/labsmooth 0.183515\ncnn/relu/adv-trained/fgsm/eps0.30_train.npy 0.132598\ncnn/relu/adv-trained/vat/eps2.10_train.npy 0.0876128\ncnn/brelu/gaussian/stdev0.30/pert-insts10 0.228296\ncnn/relu/gaussian/stdev0.30/pert-insts10 0.0665115\ncnn/relu 0.177879\n'

### Robustness as measured by proximty of decision boundary to training examples

In [10]:
from src.metrics import nearest_nieghbour_dist

params = {"eps_step": 0.05,
        "eps_max": 1.0,
        "clip_min": 0.0,
        "clip_max": 1.0}

for c,a,d in zip(CLASSIFIERS,ADV_IMAGES,range(len(DEFENSES))):
    classifier = load_classifier(join(INPUT_PATH, c),'best-weights.h5')
    avg_nn_dist = nearest_nieghbour_dist(X_test, classifier.model, X_train,  session, "fgsm", params)
    
    temp = table[d]
    temp.append(avg_nn_dist)
    table[d] = temp
    print(DEFENSES[d],avg_nn_dist)
    


'''
cnn/relu/featsqueeze1 10.3774
cnn/relu/labsmooth 10.3338
cnn/relu/adv-trained/fgsm/eps0.30_train.npy 3.93939
cnn/relu/adv-trained/vat/eps2.10_train.npy 5.41566
cnn/brelu/gaussian/stdev0.30/pert-insts10 16.9473
cnn/relu/gaussian/stdev0.30/pert-insts10 10.3566
cnn/relu 10.5697
'''

1
Feature squeezing 3.0102
Label smoothing 3.18711
FGSM Adversrial Training 4.33603
VAT 6.6475
Gaussian + BRELU  9.31665
Gaussian + RELU 10.3671
Vanilla CNN 3.90864


'\ncnn/relu/featsqueeze1 10.3774\ncnn/relu/labsmooth 10.3338\ncnn/relu/adv-trained/fgsm/eps0.30_train.npy 3.93939\ncnn/relu/adv-trained/vat/eps2.10_train.npy 5.41566\ncnn/brelu/gaussian/stdev0.30/pert-insts10 16.9473\ncnn/relu/gaussian/stdev0.30/pert-insts10 10.3566\ncnn/relu 10.5697\n'

In [11]:
print(tabulate(table,tablefmt="latex"))

\begin{tabular}{lrrr}
\hline
 Feature squeezing        & 35.61 & 0.143702 &  3.0102  \\
 Label smoothing          & 37.06 & 0.152006 &  3.18711 \\
 FGSM Adversrial Training & 56.44 & 0.226028 &  4.33603 \\
 VAT                      & 73.32 & 0.308226 &  6.6475  \\
 Gaussian + BRELU         & 69.56 & 0.471461 &  9.31665 \\
 Gaussian + RELU          & 73.96 & 0.44477  & 10.3671  \\
 Vanilla CNN              & 52.07 & 0.202975 &  3.90864 \\
\hline
\end{tabular}


In [ ]:
'''
Flips wrt: true  3561  pred  3390
 9888/10000 [============================>.] - ETA: 0sFeature squeezing 35.61
Flips wrt: true  3706  pred  3547
 9888/10000 [============================>.] - ETA: 0sLabel smoothing 37.06
Flips wrt: true  6136  pred  6027
 9888/10000 [============================>.] - ETA: 0sFGSM Adversrial Training 61.36
Flips wrt: true  8865  pred  8791
 9888/10000 [============================>.] - ETA: 0sVAT 88.65
Flips wrt: true  9073  pred  9061
 9888/10000 [============================>.] - ETA: 0sGaussian + BRELU  90.73
Flips wrt: true  9826  pred  9827
 9888/10000 [============================>.] - ETA: 0sGaussian + RELU 98.26
Flips wrt: true  5549  pred  5448
 9856/10000 [============================>.] - ETA: 0sVanilla CNN 55.49
'''

In [8]:

# get classifier
for c,a,d in zip(CLASSIFIERS,ADV_IMAGES,range(len(DEFENSES))):
    classifier = load_classifier(join(INPUT_PATH, c),'best-weights.h5')
    attack_imgs = np.load(join(WBox_ATTACKS,a,"fgsm", "eps0.10_test.npy"))
    acc = classifier.evaluate(x_val=attack_imgs,y_val=Y_test,batch_size=100,verbose=0)[1]*100
    
    temp = table[d]
    temp.append(acc)
    table[d] = temp
    print(DEFENSES[d],acc)
    
    

1
Feature squeezing 95.090000391
Label smoothing 64.4099999666
FGSM Adversrial Training 83.619999826
VAT 85.9799997211
Gaussian + BRELU  97.2000004053
Gaussian + RELU 96.680000484
Vanilla CNN 73.8800000548
